# 流程验证

### 嵌入流程

In [ ]:
chunks_path="outputs_chunks/chunk09.json"
embeddings_path="outputs_embeddings/chunk_embedding09.json"
get_embedding_then_storage(chunks_path, embeddings_path)

In [ ]:
from utils.vector_store import storage_embedding_nodes
embeddings_path="outputs_embeddings/chunk_embedding09.json"
storage_embedding_nodes(embeddings_path)


In [ ]:
# 读取数据库中的node
from utils.retrieve_nodes import get_query_engine, get_reranked_nodes
from utils.data_synthesis import save_source_nodes

question = "2022年3月1日，中国和日本进行过一次贸易战，但最终中国胜过日本。"
query_engine = get_query_engine()
source_nodes = get_reranked_nodes(question, query_engine)
source_nodes2 = [
    {
        "chunk": node.node.text,
        "score": node.score,
        "chunk_id": node.node.node_id,
        "embedding": node.node.embedding,
    } for node in source_nodes
]
save_source_nodes(question, source_nodes2, "outputs_syndatas/source_nodes01.json")
# 召回的文本块，id全都不存在于chunks中


In [ ]:
source_nodes[0].__dict__.keys()

In [ ]:
source_nodes[0].__dict__["node"].__dict__.keys()

In [ ]:
# 查看 docstore 库中的内容 查看节点
# 查看 vectorstore 库中的内容  查看向量
from llama_index.core.storage.docstore import SimpleDocumentStore
docstore_path = "llama_index/docstore02.json"
docstore = SimpleDocumentStore.from_persist_path(docstore_path)
docs = docstore.docs

In [ ]:
len(docs)

In [ ]:
docs_ls = list(docs)
docs_ls[0]

In [ ]:
docs_ls[1]


In [ ]:
docs[docs_ls[1]].node.__dict__.keys()

In [ ]:
docs[docs_ls[1]].__dict__.keys()

In [ ]:
docs[docs_ls[1]].node_id

In [ ]:
type(docs)

In [ ]:
from llama_index.core.schema import TextNode

chunk = {
    "chunk": "# 256级灰度OLED驱动电路\n陈志明,朱文清,张志林,蒋雪茵 (上海大学材料科学与工程学院,上海201800)\n摘　要：设计了一种OLED矩阵屏的驱动电路,在 $64\\times48$ 像素OLED矩阵屏上,采用脉宽调制的方法实现了256级灰度、无交叉效应的静态图案显示。通过在未选中行上施加正向偏压、未选中列上施加反向偏压的方法,有效地抑制了交叉效应。 关键词：有机电致发光；无源矩阵驱动；灰度显示\n中图分类号：TN873.1文献标识码：A 文章编号：1001-5868(2005)02-0158-03\n# 256Level Gray Scale OLED Driving Circuit\nCHEN Zhi-ming, ZHU Wen-qing, ZHANG Zhi-lin, JIANG Xue-yin (School of Materials Science and Engineering, Shanghai University, Shanghai 201800, CHN)",
    "source": "256级灰度OLED驱动电路_陈志明_llm_correct.md",
    "chunk_id": "5e96c1a3-5135-4704-b0ea-814df80d4087"
}
node = TextNode(
    text=chunk["chunk"], 
    id_=chunk["chunk_id"], 
    metadata={
        "source": chunk["source"],
    }
)
node.node_id , node.node_id == chunk["chunk_id"]

### 合成流程

In [1]:
import torch
torch.__version__

d:\Miniconda3\envs\raft\lib\site-packages\torch\nn\modules\transformer.py:20: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: 找不到指定的模块。 (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


'2.2.2'

In [2]:
import numpy as np
print(np.__version__)

2.0.1


# qwen-reranker

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

def format_instruction(instruction, query, doc):
    if instruction is None:
        instruction = "Given a query, retrieve relevant documents that can answer the query."
    output = "<Instruct>:{instruction}\n<Query>:{query}\n<Documents>:{doc}".format(instruction=instruction, query=query, doc=doc)
    return output

def process_inputs(pairs):
    inputs = tokenizer(
        pairs, padding=False, truncation="longest_first", return_attention_mask=False, 
        max_length=max_length - len(prefix_tokens) - len(suffix_tokens)
    )
    for i, ele in enumerate(inputs["input_ids"]):
        inputs["input_ids"][i] = prefix_tokens + ele + suffix_tokens
    inputs = tokenizer.pad(inputs, padding=True, return_tensors="pt", max_length=max_length)
    for key in inputs:
        inputs[key] = inputs[key].to(model.device)
    return inputs

@torch.no_grad()
def compute_logits(inputs, **kwargs):
    batch_scores = model(**inputs).logits[:, -1, :]
    true_vector = batch_scores[:, token_true_id]
    false_vector = batch_scores[:, token_false_id]
    batch_scores = torch.stack([false_vector, true_vector], dim=1)
    batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
    scores = batch_scores[:, 1].exp().tolist()
    return scores

model_name = "Qwen/Qwen3-Reranker-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, attn_implemention="flash_attention_2").cuda().eval()
token_false_id = tokenizer.convert_tokens_to_ids("no")
token_true_id = tokenizer.convert_tokens_to_ids("yes")
max_length = 2048

prefix = "<|im_start|>system\nJudge whether the document meets the requirements based on the query and the instruct provided. Note that the answer can only be yes or no.<|im_end|\n<im_start>user\n"
suffix = "<|im_end|\n<im_start>assistant\n<think>\n\n</think>\n\n"
prefix_tokens = tokenizer.encode(prefix, add_special_tokens=False)
suffix_tokens = tokenizer.encode(suffix, add_special_tokens=False)

task = "Given a query, retrieve relevant documents that answer the query."
querires = [
    "what is the capital of China?",
    "explain gravity"
]
documents = [
    "the capital of China is Beijing.",
    "gravity is a force that causes objects to fall toward the Earth's center."
]
pairs = [
    format_instruction(task, query, document) for query, document in zip(querires, documents)
]
# tokenize the input texts
inputs = process_inputs(pairs)
scores = compute_logits(inputs)
print("scores:", scores)